Generative Adversarial Networks in Keras

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Merge
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Reshape
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import random
from keras.optimizers import SGD
import pandas as pd

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [5]:
n=60

x_train=x_train[0:n]
x_test=x_test[n:n+n]

noise_factor = 0.1
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [6]:
x_train=x_train.reshape((n,28,28,1))
x_test=x_test.reshape((n,28,28,1))
x_train_noisy = x_train_noisy.reshape((n,28,28,1))
x_test_noisy = x_test_noisy.reshape((n,28,28,1))

x_train_noisy=np.concatenate([x_train_noisy,x_train]).reshape(2*n,28,28,1)
x_train=np.concatenate([x_train,x_train]).reshape(2*n,28,28,1)

In [7]:
np.random.seed(200)
sel=random.sample(range(0,x_train.shape[0]), 100)
y_train=pd.get_dummies(y_train[sel])
x_train_noisy=x_train_noisy[sel]

In [8]:
batch_size = 30
nb_classes = 10
img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)
input_shape=(28,28,1)
learning_rate = 0.008
decay_rate = 5e-5
momentum = 0.9

In [9]:
sgd = SGD(lr=learning_rate,momentum=momentum, decay=decay_rate, nesterov=False)


input_img = Input(shape=(28, 28, 1))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
generator = Model(input_img, decoded)
generator.compile(loss='mean_squared_error', optimizer=sgd,metrics = ['accuracy'])
generator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 32)        0         
__________

In [10]:
discriminator = Sequential()
discriminator.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='valid',
                        input_shape=input_shape))
discriminator.add(Activation('relu'))
discriminator.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
discriminator.add(Activation('relu'))
discriminator.add(MaxPooling2D(pool_size=pool_size))
discriminator.add(Dropout(0.25))
discriminator.add(Flatten())
discriminator.add(Dense(128))
discriminator.add(Activation('relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(10))
discriminator.add(Activation('softmax'))
discriminator.compile(loss='categorical_crossentropy', optimizer=sgd,metrics = ['accuracy'])
discriminator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In [11]:
def trainable(net, val):
    net.trainable = val
    for k in net.layers:
       k.trainable = val
trainable(generator, False)

gan_input = Input(batch_shape=(None, 28,28,1))

gan_level2 = discriminator(generator([gan_input]))

In [12]:
GAN = Model(gan_input, gan_level2)
GAN.compile(loss='mean_squared_error', optimizer=sgd,metrics = ['accuracy'])

In [13]:
nb_epochs=1000
rate=5
for i in range(0,n):
    print('Discriminator epoch', i)
    discriminator.fit(x_train_noisy,np.array(y_train),
                    epochs=nb_epochs,
                    batch_size=30,verbose=1)
    print('Generator')
    GAN.fit(x_train_noisy,np.array(y_train),
            batch_size=30, epochs=nb_epochs,verbose=1)

Discriminator epoch 0
Epoch 1/1000
100/100 [==============================] - 0s - loss: 2.3152 - acc: 0.1000     
Epoch 2/1000
100/100 [==============================] - 0s - loss: 2.3045 - acc: 0.2000     
Epoch 3/1000
100/100 [==============================] - 0s - loss: 2.2715 - acc: 0.1500     
Epoch 4/1000
100/100 [==============================] - 0s - loss: 2.2231 - acc: 0.1900     
Epoch 5/1000
100/100 [==============================] - 0s - loss: 2.1965 - acc: 0.2000     
Epoch 6/1000
100/100 [==============================] - 0s - loss: 2.2101 - acc: 0.2000     
Epoch 7/1000
100/100 [==============================] - 0s - loss: 2.1739 - acc: 0.2300     
Epoch 8/1000
100/100 [==============================] - 0s - loss: 2.1039 - acc: 0.2100     
Epoch 9/1000
100/100 [==============================] - 0s - loss: 2.0466 - acc: 0.2900     
Epoch 10/1000
100/100 [==============================] - 0s - loss: 2.0735 - acc: 0.2300     
Epoch 11/1000
100/100 [========================

KeyboardInterrupt: 

In [15]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train=y_train[0:n]
y_train=np.concatenate([y_train,y_train])[sel]
y_test=y_test[0:n]
y_test=np.concatenate([y_test,y_test])[sel]
x_test_noisy=x_test_noisy[0:n]
x_test_noisy=np.concatenate([x_test_noisy,x_test_noisy])[sel]

print('Accuracy Train:',1-np.count_nonzero(y_train-np.argmax(GAN.predict(x_train_noisy),axis=1))/n)

print('Accuracy Test:',1-np.count_nonzero(y_test-np.argmax(GAN.predict(x_test_noisy),axis=1))/n)

Accuracy Train: -0.5
Accuracy Test: -0.5
